<a id="toc"></a>

# <p style="background-color: #008080; font-family:newtimeroman; color:#FFF9ED; font-size:175%; text-align:center; border-radius:5px 5px;">Auto Scout Car Prices Prediction Project: <br> Data Cleaning</p>

## <p style="background-color: #008080; font-family:newtimeroman; color:#FFF9ED; font-size:150%; text-align:center; border-radius:10px 10px;">Content</p>

* [INTRODUCTION NOTEBOOK](00_introduction.ipynb)
* [IMPORTING LIBRARIES](#1)
* [FUNCTIONS FOR DATA IMPUTATION](#fn)
* [DATA IMPUTATION COLUMN BY COLUMN](#2)
* [THE END OF DATA IMPUTATION](#3)

<a id="1"></a>

## Importing Libraries

In [1]:
import json
import matplotlib.pyplot as plt
from itertools import product
import numpy as np
import os
import pandas as pd
from random import choices, seed
seed = 42
import re

<a id="fn"></a>

## Functions for Data Imputation

In [2]:
def fill(df, f_fill, f_use, how):
    '''
    function takes a dataframe, a feature for which missing values are to be filled, and 
    a list of features to group by, and a method from the list [median, mode].
    
    df = dataframe
    f_fill = name of feature with missing values to impute
    l_f_use = list of features based on which the missing values are to be impute, in order of decreasing importance
    '''
    
    if how == 'mode':
        uniq_f_use = df[f_use].unique()
        for u in uniq_f_use:
            if len(df[df[f_use] == u][f_fill]) > 0:
                v = df[df[f_use] == u][f_fill].mode()
                if len(v) > 0:
                    v = v[0]
                else:
                    v = float('nan')
                    print('empty class')    
            else:
                v = float('nan')
                print('empty class')
            df.loc[df[f_use] == u, f_fill] = \
                    df.loc[df[f_use] == u, f_fill].fillna(v)
        return df
    
    if how == 'median':
        uniq_f_use = df[f_use].unique()
        for u in uniq_f_use:
            if len(df[df[f_use] == u][f_fill]) > 0:
                v = df[df[f_use] == u][f_fill].median()
            else:
                v = float('nan')
                print('empty class')
            df.loc[df[f_use] == u, f_fill] = \
                    df.loc[df[f_use] == u, f_fill].fillna(v)
        return df
    
    if how == 'mean':
        uniq_f_use = df[f_use].unique()
        for u in uniq_f_use:
            if len(df[df[f_use] == u][f_fill]) > 0:
                v = df[df[f_use] == u][f_fill].mean()
            else:
                v = float('nan')
                print('empty class')
            df.loc[df[f_use] == u, f_fill] = \
                    df.loc[df[f_use] == u, f_fill].fillna(v)
        return df

<a id="2"></a>
## Data Imputation Column by Column

In [3]:
df = pd.read_json('data_post00.json', lines=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   make_model           15919 non-null  object 
 1   body_type            15859 non-null  object 
 2   price                15919 non-null  int64  
 3   km                   14895 non-null  float64
 4   prev_owner           9091 non-null   float64
 5   hp                   15831 non-null  float64
 6   type                 15917 non-null  object 
 7   first_registration   14322 non-null  float64
 8   body_color           15322 non-null  object 
 9   paint_type           10147 non-null  object 
 10  nr_doors             15707 non-null  float64
 11  nr_seats             14942 non-null  float64
 12  gearing_type         15919 non-null  object 
 13  displacement         15423 non-null  float64
 14  cylinders            10239 non-null  float64
 15  weight               8945 non-null  

### Most important columns are: make_model, type, and body_type. 
### We will start by imputing these. 

### Explore 'make_model'
* we have very few data for audi_a2 and renault_duster.
* we can drop these from the data (35 rows)

In [5]:
df.make_model.isna().sum()

0

In [6]:
df.make_model.value_counts(dropna=False)

audi_a3           3097
audi_a1           2614
opel_insignia     2598
opel_astra        2526
opel_corsa        2219
renault_clio      1839
renault_espace     991
renault_duster      34
audi_a2              1
Name: make_model, dtype: int64

In [7]:
df.drop(df[df['make_model'] == "audi_a2"].index, axis=0, inplace=True)

In [8]:
df.drop(df[df['make_model'] == "renault_duster"].index, axis=0, inplace=True)

In [9]:
df.make_model.value_counts(dropna=False)

audi_a3           3097
audi_a1           2614
opel_insignia     2598
opel_astra        2526
opel_corsa        2219
renault_clio      1839
renault_espace     991
Name: make_model, dtype: int64

In [10]:
df.make_model.isna().sum()

0

### Impute 'body_type'

* there are too many categories, we can put any less than 100 into other category
* there are 60 missing
* fill missing values by the mode of make_model
* make a function that will fill missing values of column x by mode of x by y

In [11]:
df.body_type.isna().sum()

60

In [12]:
df.body_type.value_counts(dropna=False)

sedans           7903
station wagon    3553
compact          3153
van               783
other             290
transporter        88
NaN                60
coupe              25
off-road           21
convertible         8
Name: body_type, dtype: int64

In [13]:
btv = df.body_type.value_counts()
btv

sedans           7903
station wagon    3553
compact          3153
van               783
other             290
transporter        88
coupe              25
off-road           21
convertible         8
Name: body_type, dtype: int64

In [14]:
df.body_type = [x if x not in btv[-4:] else "other" for x in df['body_type']]

In [15]:
df.body_type.value_counts()

sedans           7903
station wagon    3553
compact          3153
van               783
other             432
Name: body_type, dtype: int64

In [16]:
df['body_type'].groupby(df['make_model']).value_counts(dropna=False)

make_model      body_type    
audi_a1         sedans           1538
                compact          1039
                station wagon      21
                other              15
                van                 1
audi_a3         sedans           2598
                station wagon     282
                compact           182
                other              28
                NaN                 7
opel_astra      station wagon    1211
                sedans           1053
                compact           185
                other              70
                NaN                 7
opel_corsa      compact          1230
                sedans            875
                other             110
                NaN                 2
                van                 2
opel_insignia   station wagon    1611
                sedans            900
                other              56
                compact            27
                NaN                 3
                van 

In [17]:
# check for error
df = fill(df,'body_type','make_model','mode')

In [18]:
df.body_type.isna().sum()

0

### impute 'type'

* impute the two missing with the mode of the rest - 'used' since they have km of 115k

In [19]:
df.type.isna().sum()

2

In [20]:
df.type.value_counts(dropna=False)

used              11080
new                1632
pre_registered     1364
employees_car      1010
demonstration       796
NaN                   2
Name: type, dtype: int64

In [21]:
df[df.type.isna()].km.value_counts()

115137.0    1
Name: km, dtype: int64

In [22]:
df['type'] = df['type'].fillna('used')

In [23]:
df.type.isna().sum()

0

### Now we will start exploring the rest of the columns in order

### Impute km

* There are 1006 missing values
* I could've used first_registration, but it seems to have too many missing values
* Impute with median of the distance by a new feature which is groups by body_type (different kind of cars are used in different ways) and type (new, pre_registered, etc.)


In [24]:
df.km.isna().sum()

1006

In [25]:
df.first_registration[df.km.isna()].value_counts(dropna=False)

NaN       820
2019.0    147
2018.0     38
2017.0      1
Name: first_registration, dtype: int64

In [26]:
df.type[df.km.isna()].value_counts(dropna=False)

new               831
pre_registered    118
demonstration      33
used               16
employees_car       8
Name: type, dtype: int64

In [27]:
df.body_type[df.km.isna()].value_counts(dropna=False)

sedans           456
compact          260
station wagon    228
van               42
other             20
Name: body_type, dtype: int64

In [28]:
df.first_registration[df.km.isna()].value_counts(dropna=False)

NaN       820
2019.0    147
2018.0     38
2017.0      1
Name: first_registration, dtype: int64

In [29]:
df['for_km'] = df.groupby(['body_type', 'type']).ngroup()

In [30]:
df = fill(df,'km','for_km','median')

In [31]:
df.km.isna().sum()

0

In [32]:
df = df.drop(['for_km'], axis=1)

### Impute 'prev_owner'

* as noted in the 00_data_cleaning, missing values are for cases where there are no previous owners
* fillna with 0

In [33]:
df.prev_owner.isna().sum()

6794

In [34]:
df.prev_owner.value_counts(dropna=False)

1.0    8293
NaN    6794
2.0     778
3.0      17
4.0       2
Name: prev_owner, dtype: int64

In [35]:
 df['prev_owner'].fillna(0, inplace=True)

In [36]:
df.prev_owner.value_counts(dropna=False)

1.0    8293
0.0    6794
2.0     778
3.0      17
4.0       2
Name: prev_owner, dtype: int64

In [37]:
df.type.value_counts(dropna=False)

used              11082
new                1632
pre_registered     1364
employees_car      1010
demonstration       796
Name: type, dtype: int64

In [38]:
df.prev_owner.isna().sum()

0

### Impute hp

* impute based on mode of groups by make_model and body_type

In [39]:
df.hp.value_counts(dropna=False)

85.0     2541
66.0     2122
81.0     1401
100.0    1308
110.0    1112
         ... 
84.0        1
195.0       1
44.0        1
239.0       1
9.0         1
Name: hp, Length: 81, dtype: int64

In [40]:
df['for_hp'] = df.groupby(['make_model','body_type']).ngroup()

In [41]:
df['for_hp'].unique()
# 27 groups

array([ 2,  3,  0,  1,  4,  6,  7,  5,  8, 12,  9, 11, 10, 15, 13, 14, 16,
       20, 19, 17, 18, 21, 22, 25, 24, 23, 26, 31, 30, 28, 29, 27],
      dtype=int64)

In [42]:
df = fill(df, 'hp','for_hp','mode')

In [43]:
df.hp.isna().sum()

0

In [44]:
df = df.drop('for_hp', axis=1)

### Impute 'first_registration'

* when first_registration is missing, 'km' is very low in most cases
* I will assign 2019 to the first_registration to all missing cases of all types. I will right in all but 17 or so cases of type 'used' where mileage is higher than 500 km.



In [45]:
df['km'][df['first_registration'].isna()].describe()

count     1579.000000
mean       511.160228
std       5337.972803
min          0.000000
25%         10.000000
50%         10.000000
75%         10.000000
max      89982.000000
Name: km, dtype: float64

In [46]:
df[df['first_registration'].isna()].groupby('type')['km'].describe()

,count,mean,std,min,25%,50%,75%,max
type,,,,,,,,
demonstration,5.0,6501.400000,4232.293681,3000.0,3000.0,4307.0,11000.0,11200.0
employees_car,3.0,8700.000000,8155.979402,3500.0,4000.0,4500.0,11300.0,18100.0
new,1529.0,11.238064,20.997590,0.0,10.0,10.0,10.0,500.0
pre_registered,6.0,8.500000,3.674235,1.0,10.0,10.0,10.0,10.0
used,36.0,20313.361111,29217.062016,5.0,10.0,640.5,32604.0,89982.0


In [47]:
df['first_registration'] = df['first_registration'].fillna(2019)

In [48]:
df['first_registration'].isna().sum()

0

### impute 'body_color'

* missing body_color for disproportionately more 'new' (425/600 = 70% cases) and 'opel_insignia' + 'opel_astra' (60%)
* a little search reveals that the most popular color for cars in europe is 'black'
* 'black' is also the most popular color in this dataset
* but 'white' for many make_models if new, which seems odd. 
* I will impute the missing body_color with a corrective 'black'. 

In [49]:
df.body_color.isna().sum()

597

In [50]:
df[df.body_color.isna()]['type'].value_counts()

new               426
used              140
employees_car      15
pre_registered     11
demonstration       5
Name: type, dtype: int64

In [51]:
425/600

0.7083333333333334

In [52]:
df['type'].value_counts()

used              11082
new                1632
pre_registered     1364
employees_car      1010
demonstration       796
Name: type, dtype: int64

In [53]:
df[df.type == 'new']['body_color'].value_counts()

white     357
black     294
grey      242
blue      144
red        65
silver     63
yellow     16
green      16
brown       6
beige       2
orange      1
Name: body_color, dtype: int64

In [54]:
df[df.body_color.isna()]['make_model'].value_counts()

opel_insignia     218
opel_astra        139
audi_a1            73
audi_a3            73
renault_clio       46
opel_corsa         28
renault_espace     20
Name: make_model, dtype: int64

In [55]:
(218 + 139) / 600

0.595

In [56]:
df['make_model'].value_counts()

audi_a3           3097
audi_a1           2614
opel_insignia     2598
opel_astra        2526
opel_corsa        2219
renault_clio      1839
renault_espace     991
Name: make_model, dtype: int64

In [57]:
df.body_color.value_counts()

black     3737
grey      3504
white     3380
silver    1647
blue      1431
red        957
brown      289
green      154
beige      108
yellow      51
violet      18
bronze       6
orange       3
gold         2
Name: body_color, dtype: int64

In [58]:
df[df.type=='new'].groupby('make_model')['body_color'].value_counts(dropna=False)

make_model      body_color
audi_a1         white         118
                black          75
                NaN            36
                grey           33
                red            20
                blue           19
                yellow         15
                green          11
                silver          1
audi_a3         white          87
                grey           77
                black          75
                NaN            46
                red            14
                silver         14
                blue           13
opel_astra      NaN           108
                black          38
                white          33
                grey           31
                blue           19
                red            11
                silver         11
                green           3
                brown           1
opel_corsa      blue           44
                grey           36
                silver         22
                black

In [59]:
df['body_color'] = df['body_color'].fillna('black')

In [60]:
df[df.type=='new'].groupby('make_model')['body_color'].value_counts(dropna=False)

make_model      body_color
audi_a1         white         118
                black         111
                grey           33
                red            20
                blue           19
                yellow         15
                green          11
                silver          1
audi_a3         black         121
                white          87
                grey           77
                red            14
                silver         14
                blue           13
opel_astra      black         146
                white          33
                grey           31
                blue           19
                red            11
                silver         11
                green           3
                brown           1
opel_corsa      blue           44
                grey           36
                black          35
                silver         22
                white          10
                red             7
                yello

In [61]:
df.body_color.isna().sum()

0

### Impute 'paint_type'

* among the unique type of paints the values are only metallic (98%), basic (2%), and perlescent (rare).
* white, black, and grey body colors makes about 80% of the body_colors when paint_type is missing
* metallic is clearly the most popular finish for these body_colors even by make_model
* imputing all missing paint_type with 'Metallic'

In [62]:
df.paint_type.isna().sum()

5755

In [63]:
df.paint_type.value_counts(normalize=True)

Metallic       0.966729
Uni/basic      0.032678
Perl effect    0.000592
Name: paint_type, dtype: float64

In [64]:
df[df.paint_type.isna()].body_color.value_counts(normalize=True)

white     0.413553
black     0.261512
grey      0.123892
red       0.070895
blue      0.070721
silver    0.033362
brown     0.009904
green     0.006429
beige     0.005560
yellow    0.002085
violet    0.000869
bronze    0.000521
orange    0.000348
gold      0.000348
Name: body_color, dtype: float64

In [65]:
df[(df.body_color=='white') | (df.body_color=='black') | (df.body_color=='grey')].groupby('make_model').paint_type\
.value_counts()

make_model      paint_type 
audi_a1         Metallic       1134
                Uni/basic        38
audi_a3         Metallic       1263
                Uni/basic        85
opel_astra      Metallic        965
                Uni/basic        21
                Perl effect       2
opel_corsa      Metallic        769
                Uni/basic        42
                Perl effect       1
opel_insignia   Metallic       1179
                Uni/basic        16
renault_clio    Metallic        544
                Uni/basic        38
renault_espace  Metallic        507
                Uni/basic        16
Name: paint_type, dtype: int64

In [66]:
df['paint_type'] = df['paint_type'].fillna('Metallic')

In [67]:
df.paint_type.value_counts(dropna=False)

Metallic       15547
Uni/basic        331
Perl effect        6
Name: paint_type, dtype: int64

In [68]:
df.paint_type.isna().sum()

0

### impute 'nr_doors'

* most cars by body_type have a mode of 5 doors
* I will replace any missing with 5 doors. 
* I will also reduce the nr_doors to two classes: 5 or more and 4 or less.

In [69]:
df.nr_doors.isna().sum()

212

In [70]:
df.nr_doors.value_counts(dropna=False, normalize=True)

5.0    0.726517
4.0    0.193843
3.0    0.052380
2.0    0.013787
NaN    0.013347
1.0    0.000063
7.0    0.000063
Name: nr_doors, dtype: float64

In [71]:
df['for_doors'] = df.groupby(['make_model', 'body_type']).nr_doors.ngroup()

In [72]:
df = fill(df, 'nr_doors', 'for_doors', 'mode')

empty class


In [73]:
df.nr_doors.isna().sum()

2

In [74]:
df = fill(df, 'nr_doors', 'make_model', 'mode')

In [75]:
df = df.drop('for_doors', axis=1)

In [76]:
df.nr_doors.isna().sum()

0

### impute 'nr_seats'

* 977 missing values
* makes sense to impute the number of seats by body_type
* for all body_type other than van we can impute missing values using mode by make_model and body_type
* renault_espace is the only true 'van' in this dataset. The no. of seats are 5 or 7 in proportion 55:45
* imputing with randomly assigned 5 or 7 in proportion 55:45

In [77]:
df.nr_seats.value_counts(dropna=False)

5.0    13301
4.0     1125
NaN      977
7.0      362
2.0      116
6.0        2
3.0        1
Name: nr_seats, dtype: int64

In [78]:
df.groupby('body_type').nr_seats.value_counts(dropna=False, normalize=True)

body_type      nr_seats
compact        5.0         0.799366
               4.0         0.138193
               NaN         0.058637
               2.0         0.002536
               7.0         0.000634
               3.0         0.000317
               6.0         0.000317
other          5.0         0.766204
               2.0         0.081019
               NaN         0.078704
               7.0         0.050926
               4.0         0.023148
sedans         5.0         0.846814
               4.0         0.083785
               NaN         0.059937
               2.0         0.008959
               7.0         0.000505
station wagon  5.0         0.940780
               NaN         0.049397
               7.0         0.005333
               4.0         0.003929
               2.0         0.000281
               6.0         0.000281
van            5.0         0.475896
               7.0         0.389370
               NaN         0.132262
               2.0         0.001236
    

In [79]:
df[df.body_type=='van'].nr_seats.value_counts(dropna=False)

5.0    385
7.0    315
NaN    107
2.0      1
4.0      1
Name: nr_seats, dtype: int64

In [80]:
df[df.body_type=='van'].groupby('make_model').nr_seats.value_counts(dropna=False)

make_model      nr_seats
audi_a1         NaN           1
opel_corsa      NaN           2
opel_insignia   5.0           1
renault_clio    2.0           1
                5.0           1
renault_espace  5.0         383
                7.0         315
                NaN         104
                4.0           1
Name: nr_seats, dtype: int64

In [81]:
383/(383+315)

0.5487106017191977

In [82]:
df.loc[df.body_type != 'van', 'nr_seats'] = df.loc[df.body_type != 'van', 'nr_seats'].fillna(5)

In [83]:
df.loc[(df.body_type=='van') & (df.nr_seats.isna()),'nr_seats'] = \
    [choices((5,7), weights=(55, 45))[0] for \
     x in df.loc[(df.body_type=='van') & (df.nr_seats.isna()),'nr_seats']]

In [84]:
df[df.body_type=='van'].nr_seats.value_counts(dropna=False)

5.0    452
7.0    355
2.0      1
4.0      1
Name: nr_seats, dtype: int64

In [85]:
df.nr_seats.isna().sum()

0

### impute 'gearing_type'

In [86]:
df.gearing_type.isna().sum()

0

### impute 'displacement'

* missing in 495 rows
* displacement and cylinders are missing together in 448 cases
* the displacement medians by make_model show some trend with first_registration
* filling with medians based on groups based on 'make_model' and 'first_registration'.

In [87]:
df.displacement.isna().sum()

495

In [88]:
df[df.displacement.isna()].cylinders.value_counts(dropna=False)

NaN    448
4.0     42
3.0      2
2.0      1
5.0      1
8.0      1
Name: cylinders, dtype: int64

In [89]:
df[df.displacement.isna()].make_model.value_counts(dropna=False)

renault_clio      91
opel_astra        89
renault_espace    87
opel_insignia     75
opel_corsa        74
audi_a3           50
audi_a1           29
Name: make_model, dtype: int64

In [90]:
df.groupby(['make_model','first_registration']).displacement.describe()

count         mean         std     min  \
make_model     first_registration                                           
audi_a1        2016.0              622.0  1283.254019  244.469974   999.0   
               2017.0              424.0  1261.502358  248.676259   929.0   
               2018.0              735.0  1236.282993  252.003312   929.0   
               2019.0              804.0  1033.807214  155.925339   995.0   
audi_a3        2016.0              791.0  1635.101138  213.117932   999.0   
               2017.0              666.0  1541.614114  268.762007   999.0   
               2018.0              768.0  1539.876302  265.731612   999.0   
               2019.0              822.0  1338.698297  296.402131   997.0   
opel_astra     2016.0              498.0  1534.534137  158.972176   998.0   
               2017.0              580.0  1480.041379  194.797486   998.0   
               2018.0              692.0  1409.958092  588.597278   996.0   
               2019.0              667.0  1322.338831  246.293794   139.0   
opel_corsa     2016.0              388.0  1283.912371   78.534856   998.0   
               2017.0              374.0  1321.267380   84.783453   998.0   
               2018.0              667.0  1365.467766   68.156617   999.0   
               2019.0              716.0  1334.977654   96.593660   122.0   
opel_insignia  2016.0              555.0  1737.281081  645.575301     2.0   
               2017.0              568.0  1689.489437  180.228544  1364.0   
               2018.0              745.0  1634.920805  191.396587  1364.0   
               2019.0              655.0  1607.622901  357.203087     1.0   
renault_clio   2016.0              371.0  1350.118598  163.318300   898.0   
               2017.0              412.0  1316.128641  194.989002     1.0   
               2018.0              568.0  1289.544014  282.744963     1.0   
               2019.0              397.0   993.871537  199.743234   890.0   
renault_espace 2016.0              276.0  1601.641304   24.737861  1580.0   
               2017.0              144.0  1612.819444   49.115342  1589.0   
               2018.0              227.0  1659.039648   92.052953  1598.0   
               2019.0              257.0  1815.478599  288.120635   160.0   

                                      25%     50%     75%      max  
make_model     first_registration                                   
audi_a1        2016.0               999.0  1395.0  1422.0   1798.0  
               2017.0               999.0  1395.0  1422.0   1798.0  
               2018.0               999.0   999.0  1422.0   1798.0  
               2019.0               999.0   999.0   999.0   1984.0  
audi_a3        2016.0              1598.0  1598.0  1598.0   2480.0  
               2017.0              1598.0  1598.0  1598.0   2480.0  
               2018.0              1598.0  1598.0  1598.0   2480.0  
               2019.0               999.0  1498.0  1598.0   2480.0  
opel_astra     2016.0              1598.0  1598.0  1598.0   1686.0  
               2017.0              1399.0  1598.0  1598.0   1696.0  
               2018.0              1399.0  1399.0  1598.0  15898.0  
               2019.0               999.0  1399.0  1399.5   1600.0  
opel_corsa     2016.0              1229.0  1248.0  1398.0   1400.0  
               2017.0              1229.0  1398.0  1398.0   1400.0  
               2018.0              1398.0  1398.0  1398.0   1400.0  
               2019.0              1229.0  1398.0  1398.0   1400.0  
opel_insignia  2016.0              1598.0  1598.0  1956.0  16000.0  
               2017.0              1598.0  1598.0  1956.0   2000.0  
               2018.0              1490.0  1598.0  1600.0   2000.0  
               2019.0              1490.0  1598.0  1956.0   2000.0  
renault_clio   2016.0              1197.0  1461.0  1461.0   1618.0  
               2017.0              1149.0  1461.0  1461.0   1618.0  
               2018.0              1149.0  1197.0  1461.0   1618.0  

In [91]:
df['for_displacement'] = df.groupby(['make_model','first_registration']).displacement.ngroup()

In [92]:
df = fill(df, 'displacement', 'for_displacement', 'median')

In [93]:
df.displacement.isna().sum()

0

In [94]:
df = df.drop('for_displacement', axis=1)

### impute 'cylinders'

* research on this topic shows that apart from a few tuned up cars, we expect the cylinders in a car engine by make model to be the same.
* we can impute missing cylinder numbers based on make_model.

In [95]:
df.cylinders.value_counts(dropna=False)

4.0    8072
NaN    5678
3.0    2104
5.0      22
6.0       3
8.0       2
2.0       2
1.0       1
Name: cylinders, dtype: int64

In [96]:
df = fill(df, 'cylinders', 'make_model', 'mode')

In [97]:
df.cylinders.value_counts(dropna=False)

4.0    12891
3.0     2963
5.0       22
6.0        3
8.0        2
2.0        2
1.0        1
Name: cylinders, dtype: int64

In [98]:
df.cylinders.isna().sum()

0

### impute 'weight'

* imputing weight using make_model and first_registration

In [99]:
df.weight.isna().sum()

6939

In [100]:
df['for_weight'] = df.groupby(['make_model', 'first_registration']).ngroup()

In [101]:
df = fill(df, 'weight', 'for_weight', 'median')

In [102]:
df.weight.isna().sum()

0

In [103]:
df = df.drop('for_weight', axis=1)

### impute 'drive_chain'

* search shows that all the models described here are also available in 4WD
* first assign 'rear' drive_chain as missing.
* then impute using random assignment of front and 4WD based on proportions within class by make_model and first_registration

In [104]:
df.drive_chain.isna().sum()

6826

In [105]:
df.drive_chain.value_counts()

front    8885
4WD       169
rear        4
Name: drive_chain, dtype: int64

In [106]:
df.drive_chain = [float('nan') if x=='rear' else x for x in df.drive_chain]

In [107]:
df.groupby('make_model').drive_chain.value_counts(dropna=False)

make_model      drive_chain
audi_a1         front          1693
                NaN             918
                4WD               3
audi_a3         front          2008
                NaN            1070
                4WD              19
opel_astra      front          1346
                NaN            1178
                4WD               2
opel_corsa      front          1170
                NaN            1049
opel_insignia   front          1409
                NaN            1095
                4WD              94
renault_clio    NaN            1033
                front           795
                4WD              11
renault_espace  NaN             487
                front           464
                4WD              40
Name: drive_chain, dtype: int64

In [108]:
df['for_drivechain'] = df.groupby(['make_model', 'first_registration']).ngroup()

In [109]:
for g in df.for_drivechain.unique():
    vc = [v for v in df[df.for_drivechain==g].drive_chain.value_counts()]
    if len(vc) == 1:
        df.loc[(df.for_drivechain==g) & (df.drive_chain.isna()), 'drive_chain'] = \
            df.loc[(df.for_drivechain==g) & (df.drive_chain.isna()), 'drive_chain'].fillna('front')
    else:
        df.loc[(df.for_drivechain==g) & (df.drive_chain.isna()), 'drive_chain'] = \
            [choices(('front','4WD'), weights=vc)[0] for \
                 x in df.loc[(df.for_drivechain==g) & (df.drive_chain.isna()), 'drive_chain']]

In [110]:
df.groupby('make_model').drive_chain.value_counts(dropna=False)

make_model      drive_chain
audi_a1         front          2609
                4WD               5
audi_a3         front          3065
                4WD              32
opel_astra      front          2523
                4WD               3
opel_corsa      front          2219
opel_insignia   front          2464
                4WD             134
renault_clio    front          1812
                4WD              27
renault_espace  front           928
                4WD              63
Name: drive_chain, dtype: int64

In [111]:
df.drive_chain.isna().sum()

0

In [112]:
df = df.drop('for_drivechain', axis=1)

### impute 'co2_emission'

* impute based on mode of classes based on 'make_model', 'first_registration'

In [113]:
df.co2_emission.isna().sum()

2434

In [114]:
df[df.co2_emission.isna()].fuel.value_counts()

Diesel                                                   900
Gasoline                                                 866
Diesel (Particulate Filter)                              431
Super 95                                                 161
Super 95,Regular,Benzine 91                               17
Super 95,Super E10 95                                     14
Regular,Benzine 91                                        11
Gasoline (Particulate Filter)                              7
CNG                                                        6
LPG                                                        4
Others                                                     3
Super Plus 98                                              2
Super E10 95                                               2
Liquid petroleum gas (LPG)                                 2
Others (Particulate Filter)                                1
Regular,Benzine 91 (Particulate Filter)                    1
Super E10 95 (Particulat

In [115]:
df.groupby(['make_model','first_registration']).co2_emission.describe()

count        mean         std    min  \
make_model     first_registration                                         
audi_a1        2016.0              571.0  101.357268   11.726048   90.0   
               2017.0              368.0  103.182065    8.403775   90.0   
               2018.0              679.0  102.686303    5.930793   89.0   
               2019.0              686.0  109.297376    5.653671   92.0   
audi_a3        2016.0              679.0  106.026510   10.044703   85.0   
               2017.0              601.0  106.838602    8.835472   88.0   
               2018.0              716.0  110.736034   50.036086   36.0   
               2019.0              752.0  113.513298    8.334059   95.0   
opel_astra     2016.0              410.0  138.826829  591.751940   87.0   
               2017.0              508.0  107.001969   13.476621   88.0   
               2018.0              630.0  154.752381  674.025358   88.0   
               2019.0              539.0  126.230056   17.179756    0.0   
opel_corsa     2016.0              347.0  155.074928  655.286453   82.0   
               2017.0              325.0  122.680000    8.925743   84.0   
               2018.0              597.0  126.936348   12.048391   51.0   
               2019.0              681.0  136.535977    9.812610  101.0   
opel_insignia  2016.0              460.0  127.889130   20.348026   98.0   
               2017.0              470.0  130.363830   19.204365    0.0   
               2018.0              657.0  179.292237  776.643868    5.0   
               2019.0              512.0  144.984375   18.935459    5.0   
renault_clio   2016.0              322.0  103.813665   19.125493   82.0   
               2017.0              334.0  106.245509   19.518211   82.0   
               2018.0              500.0  118.730000   16.778562   82.0   
               2019.0              355.0  114.895775    6.805021   87.0   
renault_espace 2016.0              243.0  122.757202    7.476319  116.0   
               2017.0              119.0  130.310924   14.529775  116.0   
               2018.0              177.0  134.101695   17.169967  116.0   
               2019.0              212.0  149.995283   17.608566  116.0   

                                      25%    50%     75%      max  
make_model     first_registration                                  
audi_a1        2016.0               97.00   99.0  104.00    331.0  
               2017.0               97.00   99.5  104.25    136.0  
               2018.0               98.00  102.0  104.00    134.0  
               2019.0              108.00  110.0  111.00    142.0  
audi_a3        2016.0               99.00  101.0  114.00    189.0  
               2017.0              103.00  106.0  109.00    189.0  
               2018.0              103.75  106.0  108.00   1060.0  
               2019.0              106.75  114.0  117.00    194.0  
opel_astra     2016.0               97.00  104.0  119.00  12087.0  
               2017.0               95.00  102.0  119.00    150.0  
               2018.0              104.00  119.0  127.00  12087.0  
               2019.0              114.00  128.0  135.00    170.0  
opel_corsa     2016.0              110.00  126.0  128.00  12324.0  
               2017.0              120.00  124.0  126.00    150.0  
               2018.0              120.00  124.0  137.00    170.0  
               2019.0              129.00  139.0  143.00    167.0  
opel_insignia  2016.0              109.00  129.0  144.25    253.0  
               2017.0              114.00  134.0  144.00    197.0  
               2018.0              133.00  138.0  142.00  14457.0  
               2019.0              136.00  145.0  150.00    199.0  
renault_clio   2016.0               85.00   95.0  126.00    140.0  
               2017.0               85.00  105.0  126.00    140.0  
               2018.0              113.00  120.0  133.00    140.0  
               2019.0              113.00  113.0  118.00    140.0  
renault_espace 2

In [116]:
df['for_co2'] = df.groupby(['make_model', 'first_registration']).co2_emission.ngroup()

In [117]:
df = fill(df, 'co2_emission','for_co2', 'mode')

In [118]:
df = df.drop('for_co2', axis=1)

In [119]:
df.co2_emission.isna().sum()

0

### impute 'gears'

* based on make_model

In [120]:
df.gears.isna().sum()

4702

In [121]:
df.gears.describe()

count    11182.000000
mean         5.924432
std          0.849098
min          1.000000
25%          5.000000
50%          6.000000
75%          6.000000
max         50.000000
Name: gears, dtype: float64

In [122]:
df = fill(df,'gears','make_model', 'mode')

In [123]:
df.gears.isna().sum()

0

### impute 'country_version'

* this may hold important information
* research shows that the country versions of cars vary by the make_model and body_types. 
* I impute country_version based on mode of the 31 classes of combination of these.
* for the 9 rows in 3 empty classes I will impute these with 'Germany', by far the most common entry.


In [124]:
df.country_version.isna().sum()

8308

In [125]:
df.country_version.value_counts()

Germany           4501
Italy             1038
European Union     507
Netherlands        464
Spain              325
Belgium            313
Austria            208
Czech Republic      52
Poland              49
France              38
Denmark             33
Hungary             28
Slovakia             4
Croatia              4
Sweden               3
Romania              2
Bulgaria             2
Luxembourg           1
Switzerland          1
Slovenia             1
Egypt                1
Serbia               1
Name: country_version, dtype: int64

In [126]:
df['for_country_version'] = df.groupby(['make_model', 'body_type']).country_version.ngroup()

In [127]:
df['for_country_version'].max()

31

In [128]:
df = fill(df, 'country_version', 'for_country_version', 'mode')

empty class
empty class
empty class


In [129]:
df = df.drop('for_country_version', axis=1)

In [130]:
df.country_version.isna().sum()

9

In [131]:
df['country_version'] = df['country_version'].fillna('Germany')

In [132]:
df.country_version.isna().sum()

0

### impute 'warranty_mo'

* min vaue of warranty_mo is 1. 
* we can infer the missing value is no warranty, or 0 months

In [133]:
df.warranty_mo.isna().sum()

11063

In [134]:
df.warranty_mo.min()

1.0

In [135]:
df.warranty_mo = df.warranty_mo.fillna(0)

In [136]:
df.warranty_mo.isna().sum()

0

### impute 'upholstery_material ', 'upholstery_color', 

* I will use the mode of the class by 'body_color' for material and color, where there are clear winners
* for 1 missing classes (2 rows) I will impute using 'cloth' for material and 'black' for color, the mode for the entire data


In [137]:
df.upholstery_material.value_counts(dropna=False)

cloth           8404
NaN             4487
part leather    1499
full leather    1009
other            368
velour            60
alcantara         57
Name: upholstery_material, dtype: int64

In [138]:
df.groupby('body_color').upholstery_material.describe()

,count,unique,top,freq
body_color,,,,
beige,68,4,cloth,42
black,3118,6,cloth,2226
blue,1103,6,cloth,863
bronze,4,3,part leather,2
brown,216,6,cloth,150
gold,0,0,NaN,NaN
green,138,3,cloth,107
grey,2427,6,cloth,1658
orange,2,2,cloth,1


In [139]:
df = fill(df, 'upholstery_material', 'body_color', 'mode')

empty class


In [140]:
df['upholstery_material'] = df['upholstery_material'].fillna('cloth')

In [141]:
df.upholstery_material.isna().sum()

0

In [142]:
df.upholstery_color.value_counts(dropna=False)

black           8198
NaN             3706
grey            1360
other           1016
cloth           1005
brown            207
part leather     140
full leather     139
beige             54
velour            16
blue              16
white             13
red                9
yellow             4
orange             1
Name: upholstery_color, dtype: int64

In [143]:
df.upholstery_material.notna().sum()

15884

In [144]:
print(df.upholstery_color.isna().sum())
not_color = set(['cloth','full leather','part leather','velour'])
df['upholstery_color'] = [(np.nan if (x in not_color) else x) \
                              if x.__class__==str else x \
                                  for x in df.upholstery_color]
print(df.upholstery_color.isna().sum())

3706
5006


In [145]:
df.upholstery_color.value_counts(dropna=False)

black     8198
NaN       5006
grey      1360
other     1016
brown      207
beige       54
blue        16
white       13
red          9
yellow       4
orange       1
Name: upholstery_color, dtype: int64

In [146]:
df.upholstery_material.notna().sum()

15884

In [148]:
df.upholstery_color.isna().sum()

5006

In [149]:
df.groupby('body_color').upholstery_color.describe()

,count,unique,top,freq
body_color,,,,
beige,75,5,black,41
black,2912,9,black,2139
blue,1056,7,black,867
bronze,4,2,black,3
brown,208,5,black,159
gold,0,0,NaN,NaN
green,130,5,black,101
grey,2310,9,black,1759
orange,2,1,black,2


In [150]:
df = fill(df, 'upholstery_color', 'body_color', 'mode')

empty class


In [151]:
df.upholstery_color.isna().sum()

2

In [152]:
df['upholstery_color'] = df['upholstery_color'].fillna('black')

In [153]:
df.upholstery_color.isna().sum()

0

### impute fuel
* we need to impute fuel, before we can impute the emission class, because emission class depends of the fuel of the vehicle.
* although there are no missing values, there are too many classes in this feature
* some research allows to demystify the types of fules and classify them as gas, diesel, low_emission, and other.
* low_emission and other are very infrequent, so we can actually stick with only gas, diesel, and other.

In [154]:
df.fuel.isna().sum()

0

In [155]:
df.fuel.value_counts()

Diesel (Particulate Filter)                                                                        4314
Super 95                                                                                           3338
Gasoline                                                                                           3141
Diesel                                                                                             2984
Super 95,Regular,Benzine 91                                                                         424
                                                                                                   ... 
Regular,Benzine 91,Super 95,Regular,Benzine E10 91                                                    1
Super Plus 98,Super E10 95                                                                            1
Regular,Benzine 91,Super 95,Regular,Benzine E10 91,Super E10 95,Super Plus 98,Super Plus E10 98       1
Regular,Benzine 91,Super Plus 98,Regular,Benzine E10 91,Super 95

In [156]:
fuels = set()
for entry in df.fuel:
    for fu in entry.split(','):
        fuels.add(fu)
print(fuels)

{'Regular', 'Super 95', 'Benzine 91 (Particulate Filter)', 'Diesel', 'Super Plus 98 (Particulate Filter)', 'Super Plus E10 98 (Particulate Filter)', 'CNG', 'Domestic gas H', 'Gasoline', 'Benzine E10 91 (Particulate Filter)', 'Super 95 (Particulate Filter)', 'Super Plus E10 98', 'Electric', 'Super Plus 98', 'Diesel (Particulate Filter)', 'Super E10 95 (Particulate Filter)', 'Gasoline (Particulate Filter)', 'Liquid petroleum gas (LPG)', 'Others', 'LPG', 'Biodiesel', 'CNG (Particulate Filter)', 'Biogas', 'Benzine 91', 'Benzine E10 91', 'Others (Particulate Filter)', 'Super E10 95'}


In [157]:
fuels_dict = {
    'CNG (Particulate Filter)': 'other', 
    'LPG':'other', 
    'Domestic gas H':'other', 
    'Super 95 (Particulate Filter)': 'gas', 
    'CNG':'other', 
    'Benzine 91': 'gas', 
    'Gasoline': 'gas', 
    'Super 95': 'gas', 
    'Diesel (Particulate Filter)': 'diesel', 
    'Super Plus E10 98 (Particulate Filter)': 'gas', 
    'Benzine E10 91 (Particulate Filter)': 'gas', 
    'Others':'other', 
    'Electric':'other', 
    'Liquid petroleum gas (LPG)':'other', 
    'Gasoline (Particulate Filter)': 'gas', 
    'Super E10 95': 'gas', 
    'Others (Particulate Filter)': 'other', 
    'Diesel': 'diesel', 
    'Biogas':'other', 
    'Super Plus 98 (Particulate Filter)': 'gas', 
    'Super E10 95 (Particulate Filter)': 'gas', 
    'Benzine 91 (Particulate Filter)': 'gas', 
    'Benzine E10 91': 'gas', 
    'Regular': 'gas', 
    'Biodiesel':'other', 
    'Super Plus 98': 'gas', 
    'Super Plus E10 98': 'gas'
}

In [158]:
df['fuel'] = [','.join(set([fuels_dict[x] for x in y.split(',')])) for y in df.fuel]

In [159]:
df.fuel.value_counts()

gas          8510
diesel       7298
other          71
gas,other       5
Name: fuel, dtype: int64

In [160]:
df['fuel'] = ['other' if re.search(',',x) else x for x in df.fuel]

In [161]:
df.fuel.value_counts(dropna=False)

gas       8510
diesel    7298
other       76
Name: fuel, dtype: int64

In [162]:
df.fuel.isna().sum()

0

### impute 'emission_class'

* In Europe, emission classes, like the Euro standards, are based on the vehicle type, not the individual vehicle.
* so we can impute this based on make_model, fuel, and first_registration
* for the empty classes with can impute with 

In [163]:
df.emission_class.isna().sum()

3626

In [164]:
df['for_e_class'] = df.groupby(['make_model', 'fuel', 'first_registration']).ngroup()

In [165]:
df = fill(df, 'emission_class', 'for_e_class', 'mode')

empty class
empty class
empty class
empty class


In [166]:
df.emission_class.isna().sum()

4

In [167]:
df.emission_class.value_counts()

euro 6          12999
euro 6d-temp     2606
euro 6c           127
euro 5             78
euro 6d            62
euro 4              8
Name: emission_class, dtype: int64

In [168]:
df['emission_class'] = df['emission_class'].fillna('euro 6')

In [169]:
df = df.drop('for_e_class', axis=1)

In [170]:
df.emission_class.isna().sum()

0

### impute consumption
* represents mileage
* for the consumption variable we will impute using the mean of classes based on a  list of features in decreasing order of impportance and reliability.
* use = [make_model, body_type, fuel, first_registration],
* we gradually pop 'use' from right side if there still are missing classes. I know before hand that I can impute every row using only make_model.

In [171]:
for f in ['consumption_comb', 'consumption_city', 'consumption_country']:
    use = ['make_model','body_type', 'fuel','first_registration']
    while df[f].isna().sum() > 0:
        df['for_cons'] = df.groupby(use).ngroup()
        df = fill(df, f, 'for_cons', 'mean')
        use.pop()
    print(df[f].isna().sum())
df = df.drop('for_cons', axis=1)

0
0
0


### impute reg_cont

* reg_cont is only missing for the year 2019 months 7-12.
* we can impute with equally over 2019 + [7:12]/12

In [172]:
df[df.reg_cont.isna()].first_registration.value_counts()

2019.0    1579
Name: first_registration, dtype: int64

In [173]:
df[df['first_registration']==2019].reg_cont.value_counts(sort=False)

2019.333333    506
2019.250000    543
2019.500000    208
2019.416667    440
2019.166667    585
2019.083333    541
2019.750000      5
2019.583333      6
2019.666667      1
2019.916667      1
2020.000000      1
Name: reg_cont, dtype: int64

In [174]:
df.loc[df.reg_cont > 2019.5, 'reg_cont'] = float('nan')

In [175]:
c = [2019+7/12,2019+8/12,2019+9/12,2019+10/12,2019+11/12,2019+12/12]
df.loc[(df['first_registration']==2019) & (df.reg_cont.isna()),'reg_cont'] = \
        [choices(c) for x in df.loc[(df['first_registration']==2019) & (df.reg_cont.isna()),'reg_cont']]

In [176]:
df.reg_cont.isna().sum()

0

In [177]:
df[df['first_registration']==2019].reg_cont.value_counts(dropna=False)

2019.166667    585
2019.250000    543
2019.083333    541
2019.333333    506
2019.416667    440
2019.583333    283
2019.916667    281
2019.833333    273
2019.666667    267
2020.000000    245
2019.750000    244
2019.500000    208
Name: reg_cont, dtype: int64

## explore features that are a list of features 

### explore comfort_convenience, entertainment_media, extras, safety_security

* since these are going to be encoded using One Hot encoder and not used as such, the missing values are not a concern. 
* although the frequency of occurence of many of the features in these are very low, these may be important predictors of price, esp if an expensive feature such as TV, or an outstanding one such as right hand drive. 
* I am not dropping any of these right away.

In [178]:

for f in ['comfort_convenience', 'entertainment_media', 'extras', 'safety_security']:
    vc = df[f].value_counts()
    vci = [i for i in vc.index]
    cc = set()
    count = {}
    for x in vci:
        for y in x.split(','):
            cc.add(y)
            if y in count:
                count[y] += 1
            else:
                count[y] = 1
    print(f)
    count = [(count[k], k) for k in count.keys()]
    count.sort(key= lambda i: i[0])
    count = [str(k[0])+'\t'+k[1] for k in count]
    print('\n'.join(count))
    print('---')

comfort_convenience
1	Electric Starter
11	Windshield
41	Wind deflector
48	Leather seats
59	Air suspension
208	Auxiliary heating
265	Massage seats
308	Sunroof
431	Panorama roof
455	Heads-up display
527	Seat ventilation
528	Parking assist system self-steering
574	Electrically heated windshield
606	Electric tailgate
1018	Tinted windows
1067	Electrically adjustable seats
1358	Heated steering wheel
1401	Keyless central door lock
1550	Split rear seats
1659	Lumbar support
1719	Parking assist system camera
2809	Hill Holder
2969	Parking assist system sensors front
3199	Seat heating
3485	Light sensor
3522	Armrest
3927	Start-stop system
3998	Navigation system
4007	Rain sensor
4029	Automatic climate control
4380	Leather steering wheel
4589	Park Distance Control
4616	Parking assist system sensors rear
4747	Cruise control
4865	Multi-function steering wheel
5080	Electrical side mirrors
5826	Power windows
5977	Air conditioning
---
entertainment_media
18	Television
133	Digital radio
138	Sound system
15

## Save intermediate file

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15884 entries, 0 to 15918
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   make_model           15884 non-null  object 
 1   body_type            15884 non-null  object 
 2   price                15884 non-null  int64  
 3   km                   15884 non-null  float64
 4   prev_owner           15884 non-null  float64
 5   hp                   15884 non-null  float64
 6   type                 15884 non-null  object 
 7   first_registration   15884 non-null  float64
 8   body_color           15884 non-null  object 
 9   paint_type           15884 non-null  object 
 10  nr_doors             15884 non-null  float64
 11  nr_seats             15884 non-null  float64
 12  gearing_type         15884 non-null  object 
 13  displacement         15884 non-null  float64
 14  cylinders            15884 non-null  float64
 15  weight               15884 non-null 

In [180]:
df.to_json('data_post01.json', orient='records', lines=True)

## Summary

* in this part, I observed each feature and replaced missing values with plausible values. 
* I also dropped 36 rows of data for make_model audi_a2 (n=1) and ranault_duster (n=35)
* for fuel had many features nested inside it, I replaced these with the most meaningful ones.
* for comfort_convenience, entertainment_media, extras, safety_security I left these features as they are.

<a id="3"></a>

## End of Data Imputation

<a href="#toc" class="btn btn-primary btn-sm" role="button" aria-pressed="true" 
style="color:blue; background-color:#dfa8e4" data-toggle="popover">Content</a>

## Next: [Handling outliers](03_data_viz_&_outliers.ipynb)